##统计不同户型，在特定年代范围的二手房平均价格折线分布图##
#已知：#
* 二手房户型，如：3室2厅2卫
* 年代范围，如：1990-2017等
#求：#
* 不同户型不同年代的房价折线分布图，如：3室2厅2卫的二手房价格在1999，2005，2010年代的平均价格是多少

In [2]:
import pymongo
import charts

Server running in the folder C:\Users\Administrator\Desktop\ganjiFang at 127.0.0.1:52263


In [3]:
client = pymongo.MongoClient('localhost', 27017)
ganjiCrawler = client['ganjiCrawler']
fangInfo = ganjiCrawler['fangInfo']

In [4]:
# series: [{
#         name: '东京',
#         data: [7.0, 6.9, 9.5, 14.5, 18.2, 21.5, 25.2, 26.5, 23.3, 18.3, 13.9, 9.6]
#     }, {
#         name: '纽约',
#         data: [-0.2, 0.8, 5.7, 11.3, 17.0, 22.0, 24.8, 24.1, 20.1, 14.1, 8.6, 2.5]
#     }, {
#         name: '柏林',
#         data: [-0.9, 0.6, 3.5, 8.4, 13.5, 17.0, 18.6, 17.9, 14.3, 9.0, 3.9, 1.0]
#     }, {
#         name: '伦敦',
#         data: [3.9, 4.2, 5.7, 8.5, 11.9, 15.2, 17.0, 16.6, 14.2, 10.3, 6.6, 4.8]
#     }]

#Step1：求共有哪几种不同的户型
huxingList = [i['huxing'] for i in fangInfo.find()]
huxingSetList = list(set(huxingList))

In [5]:
#更户型数量小于30的，更新为'其他’户型
for huxing in huxingSetList:
    count = fangInfo.find({'huxing':huxing}).count()
    if count < 30:
        fangInfo.update_many({'huxing':huxing},{'$set':{'huxing':'其他'}})
    else:
        pass
print('ok')

ok


In [6]:
#统计户型数量，将户型数量少于30的户型归为“其他”户型，得出新的户型列表
# def huxingFenLei():
#     huxingNewList = []
#     for huxing in huxingSetList:
#         count = huxingList.count(huxing)
#         if count < 30:
#             pass
#         else:
#             huxingNewList.append(huxing)
#     huxingNewList.append('其他')
#     return huxingNewList

# huxingNewList = huxingFenLei()
# print(len(huxingNewList))

# #更新完户型信息后，重新统计户型种类个数
huxingList = [i['huxing'] for i in fangInfo.find()]
huxingSetList = list(set(huxingList))
print(len(huxingSetList))

13


In [7]:
#Step2：统计不同户型，在1990-2017年每年的平均房价

#整理更新fangInfo中niandai的数据
for i in fangInfo.find():
    yearNew = i['niandai'][:4]
    fangInfo.update_one({'_id':i['_id']},{'$set':{'niandai':yearNew}})
print('ok')

ok


In [8]:
#更新fangInfo表中price，将其设置为int格式
for i in fangInfo.find():
    priceNew = int(float(i['price']))
    fangInfo.update_one({'_id':i['_id']},{'$set':{'price':priceNew}})
print('ok')

ok


In [9]:
#求在1990-2017年中每一年不同户型的平均房价
#         {'$group':{'_id':'$niandai', 'avg_price':{'$avg':'$price'}}},
#         {'$sort':{'avg_price':1}}
def avgPrice(date1, date2, huxing):
    pipeline = [
        {'$match':{'$and':[{'niandai':{'$gte':date1, '$lte':date2}}, {'huxing':huxing}]}}, 
        {'$group':{'_id':'$niandai', 'avg_price':{'$avg':'$price'}}},{'$sort':{'_id':1}}]
    for i in fangInfo.aggregate(pipeline):
        yield i['avg_price']

In [10]:
#根据图表数据结构，生成所有数据
def getAllData(date1,date2,huxingList):
    series = []
    for huxing in huxingList:
        data = {
            'name':huxing,
            'data':[i for i in avgPrice(date1,date2,huxing)]
        }
        series.append(data)
#         print('{}:{}'.format(huxing,len(data['data'])))
    return series


In [12]:
options = {
    'chart':{'zoomType':'xy'},
    'title':{'text':'广州各户型二手房在1990-2017年的平均价格变化图'},
    'xAxis':{'categories':[str(i) for i in range(1990,2018)],'title':{'text':'年代'}},
    'yAxis':{'title':{'text':'价格（万元）'}, 'min':0},
    'plotOptions':{'line':{'dataLabels':{'enabled':'true', 'color':'#000000'}}}
}

series = getAllData('1990', '2017', huxingSetList)

charts.plot(series, show='inline', options=options)